In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

In [46]:
df = pd.read_csv('Train_TEC_mar17.csv')

In [47]:
df.head()

,Date,Month,Year,Time,LT,TEC
0,1,3,2017,0.016389,5.311333,2.067562
1,1,3,2017,0.049722,5.344670,2.082707
2,1,3,2017,0.083056,5.378006,2.004272
3,1,3,2017,0.116389,5.411343,1.736569
4,1,3,2017,0.149722,5.444679,1.758503


In [48]:
df.drop(["Date", "Month", "Year", "LT","Time"], axis = 1, inplace = True)

In [49]:
df.head()

,TEC
0,2.067562
1,2.082707
2,2.004272
3,1.736569
4,1.758503


In [50]:
type(df)

pandas.core.frame.DataFrame

In [51]:
df

,TEC
0,2.067562
1,2.082707
2,2.004272
3,1.736569
4,1.758503
...,...
14395,1.717251
14396,1.653964
14397,1.315151
14398,1.293472


In [53]:
train, test = df[:-1440], df[-1400:]

In [54]:
train

,TEC
0,2.067562
1,2.082707
2,2.004272
3,1.736569
4,1.758503
...,...
12955,1.306469
12956,1.408271
12957,1.135789
12958,0.969443


In [55]:
test

,TEC
13000,7.192375
13001,7.715456
13002,7.550865
13003,7.518970
13004,7.713859
...,...
14395,1.717251
14396,1.653964
14397,1.315151
14398,1.293472


In [56]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [59]:
n_input = 2
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=3)
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit_generator(generator,epochs=25)

Epoch 1/25
4320/4320 [==============================] - 20s 5ms/step - loss: 0.0021
Epoch 2/25
4320/4320 [==============================] - 20s 5ms/step - loss: 5.6637e-04
Epoch 3/25
4320/4320 [==============================] - 19s 4ms/step - loss: 4.9061e-04
Epoch 4/25
4320/4320 [==============================] - 20s 5ms/step - loss: 4.8029e-04
Epoch 5/25
4320/4320 [==============================] - 16s 4ms/step - loss: 4.5275e-04
Epoch 6/25
4320/4320 [==============================] - 12s 3ms/step - loss: 4.5208e-04
Epoch 7/25
4320/4320 [==============================] - 12s 3ms/step - loss: 4.3982e-04
Epoch 8/25
4320/4320 [==============================] - 12s 3ms/step - loss: 4.2908e-04
Epoch 9/25
4320/4320 [==============================] - 12s 3ms/step - loss: 4.3145e-04
Epoch 10/25
4320/4320 [==============================] - 12s 3ms/step - loss: 4.2793e-04
Epoch 11/25
4320/4320 [==============================] - 12s 3ms/step - loss: 4.2128e-04
Epoch 12/25
4320/4320 [===========

In [62]:
pred_list = []

batch = train[-2:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [63]:
pred_list

[array([0.02492985], dtype=float32), array([0.02754578], dtype=float32)]

In [44]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list), index=df.iloc[-n_input:].index, columns=['Prediction'])
df_test = pd.concat([df,df_predict], axis=1)
df.shape, df_predict.shape

ValueError: Shape of passed values is (272344, 2), indices imply (14400, 2)